In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np


# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns


# Ignore warnings (useful during development to reduce console noise)
import warnings
warnings.filterwarnings("ignore")


# TensorFlow and Keras for deep learning and recommendation systems
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow_recommenders as tfrs
from tensorflow.keras import metrics as keras_metrics


# Scikit-learn for preprocessing, model evaluation, and splitting
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer

# Other utility libraries
import tempfile  # For creating temporary directories/files
import os
import math
from collections import defaultdict # For handling dictionary-like structures with default values

from keras.saving import register_keras_serializable
from tensorflow.keras.models import load_model  # To load a saved Keras model
from keras.saving import register_keras_serializable  # To register custom Keras components for saving/loading
import pickle  # For serializing Python objects (like models or encoders)

c:\Program Files\Python312\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [16]:
class EnhancedMovieDataProcessor:
    def __init__(self, df):
        self.df = df.copy()
        self._preprocess_data()

    def _preprocess_data(self):
        """Enhanced data cleaning and preparation"""
        # Robust missing value handling
        self.df["movies_avg_rating"] = (
            self.df["movies_avg_rating"]
            .fillna(self.df.groupby("movieId")["movies_avg_rating"].transform("mean"))
            .fillna(self.df["movies_avg_rating"].mean())
        )

        self.df["movie_year"] = (
            self.df["movie_year"]
            .fillna(self.df.groupby("movieId")["movie_year"].transform("median"))
            .fillna(self.df["movie_year"].median())
        )

        # Enhanced user statistics
        self.df["user_rating_count"] = self.df.groupby("userId")["user_rating"].transform("count")
        self.df["user_rating_avg"] = self.df.groupby("userId")["user_rating"].transform("mean")
        self.df["user_rating_std"] = (
            self.df.groupby("userId")["user_rating"].transform("std").fillna(0)
        )

        # Enhanced genre processing
        if "genres" in self.df.columns and "Action" not in self.df.columns:
            self._process_genre_features()

        # Enhanced text features
        if not any(col.startswith("title_") for col in self.df.columns):
            self._create_text_features()

        # Additional temporal features
        if "timestamp" in self.df.columns:
            self._create_temporal_features()

    def _process_genre_features(self):
        """More robust genre processing"""
        genres = self.df["genres"].str.get_dummies(sep="|")
        # Remove rare genres (appearing in less than 1% of movies)
        genre_counts = genres.sum()
        common_genres = genre_counts[genre_counts > len(self.df) * 0.01].index
        genres = genres[common_genres]
        self.df = pd.concat([self.df, genres], axis=1)

    def _create_text_features(self):
        """Enhanced text feature extraction"""
        tfidf = TfidfVectorizer(
            stop_words="english",
            max_features=100,  # Increased features
            ngram_range=(1, 2),  # Include bigrams
            min_df=5,  # Ignore very rare terms
        )
        # Clean titles by removing year information if present
        clean_titles = (
            self.df["movie_title"].str.replace(r"\(\d{4}\)", "", regex=True).str.strip()
        )
        title_features = tfidf.fit_transform(clean_titles)
        title_df = pd.DataFrame(
            title_features.toarray(),
            columns=[f"title_{i}" for i in range(title_features.shape[1])],
            index=self.df.index,
        )
        self.df = pd.concat([self.df, title_df], axis=1)
        self.title_feature_columns = title_df.columns.tolist()

    def _create_temporal_features(self):
        """Extract temporal patterns from timestamps"""
        self.df["timestamp"] = pd.to_datetime(self.df["timestamp"], unit="s")
        self.df["rating_year"] = self.df["timestamp"].dt.year
        self.df["rating_month"] = self.df["timestamp"].dt.month
        self.df["rating_day"] = self.df["timestamp"].dt.day
        self.df["rating_hour"] = self.df["timestamp"].dt.hour
        self.df["rating_dayofweek"] = self.df["timestamp"].dt.dayofweek

    def get_user_features(self):
        """Enhanced user feature engineering"""
        user_features = (
            self.df.groupby("userId")
            .agg({
                "user_rating_count": "first",
                "user_rating_avg": "first",
                "user_rating_std": "first",
            })
            .reset_index()
        )

        # Calculate genre preferences with more robust weighting
        genre_cols = [
            col
            for col in self.df.columns
            if col.startswith(tuple(["Action", "Adventure", "Animation"]))
        ]  # Add all genre prefixes

        filtered_df = self.df[
            self.df["user_rating"].notna() & (self.df["user_rating"] > 0)
        ].copy()

        for genre in genre_cols:
            # Weighted average considering rating confidence
            genre_pref = filtered_df.groupby("userId").apply(
                lambda x: np.average(
                    x[genre],
                    weights=x["user_rating"] * np.log1p(x["user_rating_count"]),
                    axis=0,
                )
            )
            user_features = user_features.merge(
                genre_pref.rename(f"pref_{genre}").reset_index(),
                on="userId",
                how="left",
            )

        # Fill NA values with global averages
        pref_cols = [f"pref_{genre}" for genre in genre_cols]
        user_features[pref_cols] = user_features[pref_cols].fillna(
            user_features[pref_cols].mean()
        )

        return user_features.set_index("userId")

    def get_movie_features(self):
        """Enhanced movie feature engineering"""
        movie_features = self.df.groupby("movieId").agg({
            "movies_avg_rating": "first",
            "movie_year": "first",
            "movie_title": "first",
            "genres": "first",
        })

        # Add enhanced genre indicators
        genre_cols = [
            col
            for col in self.df.columns
            if col.startswith(tuple(["Action", "Adventure"]))
        ]  # Add all genre prefixes

        for genre in genre_cols:
            movie_features[genre] = self.df.groupby("movieId")[genre].max()

        # Add enhanced title embeddings
        title_cols = [col for col in self.df.columns if col.startswith("title_")]
        for col in title_cols:
            movie_features[col] = self.df.groupby("movieId")[col].mean()

        # Add popularity features
        movie_features["rating_count"] = self.df.groupby("movieId").size()
        movie_features["rating_recency"] = (
            self.df.groupby("movieId")["timestamp"].max()
            if "timestamp" in self.df.columns
            else 0
        )

        return movie_features

In [ ]:
@register_keras_serializable(package="CustomRecommenderModels")
class EnhancedRecommenderModel(tfrs.models.Model):
    def __init__(
        self,
        user_features_shape,
        movie_features_shape,
        embedding_dim=64,  # Reduced from 128
        name="EnhancedRecommenderModel",
        **kwargs,
    ):
        super().__init__(name=name, **kwargs)
        self.user_features_shape = user_features_shape
        self.movie_features_shape = movie_features_shape
        self.embedding_dim = embedding_dim

        # Simplified user model
        self.user_model = tf.keras.Sequential(
            [
                layers.Input(shape=(user_features_shape,)),
                layers.Dense(256, activation="relu"),
                layers.Dropout(0.3),
                layers.Dense(embedding_dim),
            ],
            name="user_model",
        )

        # Simplified movie model
        self.movie_model = tf.keras.Sequential(
            [
                layers.Input(shape=(movie_features_shape,)),
                layers.Dense(256, activation="relu"),
                layers.Dropout(0.3),
                layers.Dense(embedding_dim),
            ],
            name="movie_model",
        )

        # More focused rating model
        self.rating_model = tf.keras.Sequential(
            [
                layers.Dense(128, activation="relu"),
                layers.Dense(64, activation="relu"),
                layers.Dense(1),
            ],
            name="rating_model",
        )

        # Enhanced task configuration
        self.task = tfrs.tasks.Ranking(
            loss=losses.MeanSquaredError(),  # Changed from Huber
            metrics=[
                keras_metrics.RootMeanSquaredError(name="rmse"),
                keras_metrics.MeanAbsoluteError(name="mae"),
            ],
        )

    def call(self, features):
        user_embeddings = self.user_model(features["user_features"])
        movie_embeddings = self.movie_model(features["movie_features"])

        # Add attention mechanism
        attention = tf.nn.softmax(
            tf.reduce_sum(
                tf.multiply(user_embeddings, movie_embeddings), axis=1, keepdims=True
            )
        )

        concatenated = tf.concat(
            [
                user_embeddings,
                movie_embeddings,
                user_embeddings * movie_embeddings,
                attention * user_embeddings,
                attention * movie_embeddings,
            ],
            axis=1,
        )

        return self.rating_model(concatenated)

    def compute_loss(self, features, training=False):
        features_copy = features.copy()
        ratings = features_copy.pop("user_rating")
        rating_predictions = self(features_copy)

        # Add L2 regularization loss
        reg_loss = sum(self.losses)
        task_loss = self.task(labels=ratings, predictions=rating_predictions)

        return task_loss + 0.01 * reg_loss

    def get_config(self):
        config = super().get_config()
        config.update({
            "user_features_shape": self.user_features_shape,
            "movie_features_shape": self.movie_features_shape,
            "embedding_dim": self.embedding_dim,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [14]:
class EnhancedRecommenderSystem:
    def __init__(self, data_processor):
        self.data_processor = data_processor
        self.user_features = None
        self.movie_features = None
        self.model = None
        self.scalers = {
            "user": StandardScaler(),
            "movie": StandardScaler(),
            "rating": MinMaxScaler((-1, 1)),
        }
        self.imputers = {
            "user": SimpleImputer(strategy="mean"),
            "movie": SimpleImputer(strategy="mean"),
        }
        self.user_cols = None
        self.movie_cols = None
        self.similarity_matrix = None
        self.original_test_df = None 

    def recommend_for_user(self, user_id, top_n=10, diversity=0.2):
        """Robust recommendation generation with proper error handling and diversity"""
        try:
            # Check if user exists in our features
            if self.user_features is None or user_id not in self.user_features.index:
                print(f"Error generating recommendations for user {user_id}: {str(e)}")
                return self.get_popular_recommendations(top_n)

            # Get properly formatted user features
            user_data = pd.DataFrame(self.user_features.loc[user_id]).T
            user_data = self.imputers["user"].transform(user_data[self.user_cols])
            user_data_scaled = self.scalers["user"].transform(user_data)

            # Prepare all movie features
            movie_data = self.movie_features.drop(
                ["movie_title", "genres"], axis=1, errors="ignore"
            )
            expected_movie_cols = self.scalers["movie"].feature_names_in_
            movie_data = movie_data.reindex(columns=expected_movie_cols, fill_value=0)
            movie_data = self.imputers["movie"].transform(movie_data)
            movie_data_scaled = self.scalers["movie"].transform(movie_data)
            movie_data_scaled = np.nan_to_num(movie_data_scaled, nan=0.0)

            # Batch predictions to avoid memory issues
            batch_size = 1000
            predictions = []
            for i in range(0, len(movie_data_scaled), batch_size):
                batch = movie_data_scaled[i:i+batch_size]
                user_repeated = np.repeat(user_data_scaled, len(batch), axis=0)
                batch_pred = self.model({
                    "user_features": user_repeated,
                    "movie_features": batch
                }).numpy().flatten()
                predictions.extend(batch_pred)
            
            predictions = np.array(predictions)
            
            # Add some randomness for diversity
            if diversity > 0:
                noise = np.random.normal(0, diversity * np.std(predictions), len(predictions))
                predictions = predictions + noise

            # Get top N recommendations
            top_indices = np.argsort(-predictions)[:top_n*3]  # Get more candidates
            candidates = self.movie_features.iloc[top_indices].copy()
            candidates['predicted_rating'] = predictions[top_indices]
            
            # Deduplicate and diversify by genres
            final_recs = []
            seen_genres = set()
            for _, row in candidates.iterrows():
                movie_genres = set(row['genres'].split('|')) if pd.notna(row['genres']) else set()
                
                # Skip if too similar to already selected movies
                if len(seen_genres & movie_genres) / len(movie_genres) > 0.7:
                    continue
                    
                final_recs.append({
                    "movieId": row.name,
                    "title": row.get("movie_title", "N/A"),
                    "genres": row.get("genres", ""),
                    "predicted_rating": float(row['predicted_rating']),
                    "average_rating": row.get("movies_avg_rating", "N/A"),
                })
                seen_genres.update(movie_genres)
                
                if len(final_recs) >= top_n:
                    break
                    
            return pd.DataFrame(final_recs)

        except Exception as e:
            print(f"Error generating recommendations for user {user_id}: {str(e)}")
            return self.get_popular_recommendations(top_n)

    def get_popular_recommendations(self, top_n=10):
        """Fallback to popular items when personalized recs fail"""
        try:
            if self.movie_features is None:
                return pd.DataFrame([{
                    "movieId": -1,
                    "title": "No recommendations available",
                    "genres": "",
                    "year": "Unknown",
                    "predicted_rating": 0,
                    "average_rating": "N/A",
                    "popularity": 0,
                }])
                
            popular = self.movie_features.sort_values(
                by=["rating_count", "movies_avg_rating"], ascending=[False, False]
            ).head(top_n)

            return pd.DataFrame(
                [
                    {
                        "movieId": idx,
                        "title": row.get("movie_title", "N/A"),
                        "genres": row.get("genres", ""),
                        "year": row.get("movie_year", "Unknown"),
                        "predicted_rating": row.get("movies_avg_rating", 0),
                        "average_rating": row.get("movies_avg_rating", "N/A"),
                        "popularity": row.get("rating_count", 0),
                    }
                    for idx, row in popular.iterrows()
                ]
            )
        except Exception as e:
            print(f"Error in popular recommendations: {str(e)}")
            return pd.DataFrame([{
                "movieId": -1,
                "title": "No recommendations available",
                "genres": "",
                "year": "Unknown",
                "predicted_rating": 0,
                "average_rating": "N/A",
                "popularity": 0,
            }])

    def get_similar_movies(self, movie_id, top_n=10, method="hybrid"):
        """Enhanced similarity with hybrid approach"""
        try:
            if self.movie_features is None or movie_id not in self.movie_features.index:
                raise ValueError("Movie not found in database")

            # Use precomputed similarity matrix if available
            if self.similarity_matrix is not None:
                target_idx = self.movie_features.index.get_loc(movie_id)
                sim_scores = list(enumerate(self.similarity_matrix[target_idx]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
                sim_scores = sim_scores[1:top_n+1]
                movie_indices = [i[0] for i in sim_scores]
                similar = self.movie_features.iloc[movie_indices].copy()
                similar["similarity_score"] = [i[1] for i in sim_scores]
            else:
                # Content-based similarity
                movie_data = self.movie_features.drop(
                    ["movie_title", "genres"], axis=1, errors="ignore"
                )
                expected_cols = self.scalers["movie"].feature_names_in_
                movie_data = movie_data.reindex(columns=expected_cols, fill_value=0)
                movie_data = self.imputers["movie"].transform(movie_data)
                movie_data_scaled = self.scalers["movie"].transform(movie_data)
                movie_data_scaled = np.nan_to_num(movie_data_scaled, nan=0.0)

                target_idx = self.movie_features.index.get_loc(movie_id)
                content_sim = cosine_similarity(
                    movie_data_scaled[target_idx : target_idx + 1], movie_data_scaled
                )[0]

                if method == "content":
                    similar_indices = np.argsort(-content_sim)[1 : top_n + 1]
                else:
                    # Hybrid approach - combine content and collaborative
                    # Get user-item matrix for collaborative filtering
                    user_item = self.data_processor.df.pivot_table(
                        index="userId",
                        columns="movieId",
                        values="user_rating",
                        fill_value=0,
                    )

                    # Compute item-item similarity
                    item_sim = cosine_similarity(user_item.T)
                    collab_sim = item_sim[self.movie_features.index.get_loc(movie_id)]

                    # Combine similarities
                    combined_sim = 0.7 * content_sim + 0.3 * collab_sim
                    similar_indices = np.argsort(-combined_sim)[1 : top_n + 1]

                similar = self.movie_features.iloc[similar_indices].copy()
                similar["similarity_score"] = content_sim[similar_indices]

            similar_movies = similar[
                [
                    "movie_title",
                    "genres",
                    "movie_year",
                    "movies_avg_rating",
                    "rating_count",
                    "similarity_score",
                ]
            ]
            
            # Sort by similarity score
            similar_movies = similar_movies.sort_values("similarity_score", ascending=False)
            
            return similar_movies

        except Exception as e:
            print(f"Error finding similar movies: {e}")
            return pd.DataFrame()

    
    @classmethod
    def load_for_deployment(cls, data_processor, base_path="recommender_deployment"):
        """Load a saved recommender system for deployment"""

        # Initialize an empty recommender system with the provided data processor
        recommender = cls(data_processor)

        # Load scalers and imputers from a pickle file
        with open(os.path.join(base_path, "scalers_imputers.pkl"), 'rb') as f:
            saved_data = pickle.load(f)
            recommender.scalers = saved_data['scalers']
            recommender.imputers = saved_data['imputers']
            recommender.user_cols = saved_data['user_cols']
            recommender.movie_cols = saved_data['movie_cols']
            recommender.user_features = saved_data['user_features']  # Load user features

        # Load model architecture and weights
        user_features_shape = len(recommender.user_cols)
        movie_features_shape = len(recommender.movie_cols)

        recommender.model = EnhancedRecommenderModel(
            user_features_shape, 
            movie_features_shape
        )

        # Dummy call to build the model
        dummy_input = {
            'user_features': np.zeros((1, user_features_shape)),
            'movie_features': np.zeros((1, movie_features_shape))
        }

        _ = recommender.model(dummy_input)

        # Load weights
        recommender.model.load_weights(os.path.join(base_path, "model_weights.weights.h5"))

        # Load similarity matrix if available
        similarity_path = os.path.join(base_path, "similarity_matrix.npy")
        if os.path.exists(similarity_path):
            recommender.similarity_matrix = np.load(similarity_path)

        # Load movie metadata
        metadata_path = os.path.join(base_path, "movie_metadata.csv")
        if os.path.exists(metadata_path):
            recommender.movie_features = pd.read_csv(metadata_path, index_col=0)

        # Load full movie features if available
        try:
            recommender.movie_features = data_processor.get_movie_features()
        except Exception as e:
            print(f"Could not load full movie features: {str(e)}")

        return recommender

## 5. Development Architecture

In [3]:
df = pd.read_csv("df1.csv")

In [4]:
# Take the sample from the df 
df2 = df[df["user_rating_count"] > 30]

In [5]:
data_processor = EnhancedMovieDataProcessor(df2)

In [15]:
loaded_recommender = EnhancedRecommenderSystem.load_for_deployment(
    data_processor, "my_recommender_deployment"
)

In [22]:
user_id = 174862  
recommendations = loaded_recommender.recommend_for_user(user_id ,top_n=5)
recommendations

,movieId,title,genres,predicted_rating,average_rating
0,141745,Nice Guy,(no genres listed),12.058871,5.00
1,205293,Polar Bear: Spy on the Ice,Documentary,11.949726,4.50
2,212573,Sala samobójców. Hejter,Drama|Romance|Thriller,11.912901,4.75
3,177145,The Death Collector,Crime,11.611368,5.00
4,7485,Good Morning (Ohayô),Comedy,11.558308,4.36


In [23]:
loaded_recommender.get_popular_recommendations()

,movieId,title,genres,year,predicted_rating,average_rating,popularity
0,1721,Titanic,Drama|Romance,1997.0,3.45,3.45,125
1,260,Star Wars: Episode IV - A New Hope,Action|Adventure|Sci-Fi,1977.0,4.11,4.11,123
2,2959,Fight Club,Action|Crime|Drama|Thriller,1999.0,4.22,4.22,121
3,2571,"Matrix, The",Action|Sci-Fi|Thriller,1999.0,4.17,4.17,119
4,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1981.0,4.12,4.12,118
5,1265,Groundhog Day,Comedy|Fantasy|Romance,1993.0,3.89,3.89,118
6,1036,Die Hard,Action|Crime|Thriller,1988.0,3.95,3.95,116
7,858,"Godfather, The",Crime|Drama,1972.0,4.32,4.32,114
8,8368,Harry Potter and the Prisoner of Azkaban,Adventure|Fantasy|IMAX,2004.0,3.81,3.81,113
9,541,Blade Runner,Action|Sci-Fi|Thriller,1982.0,4.13,4.13,112


In [ ]:
import gradio as gr
import requests
from urllib.parse import quote

# TMDB API Configuration
TMDB_API_KEY = "c4b76d5deff8af9434b07dde68c7c158"
TMDB_BASE_URL = "https://api.themoviedb.org/3"
POSTER_BASE_URL = "https://image.tmdb.org/t/p/w200"

def get_movie_poster(movie_title, year=None):
    """Fetch movie poster URL from TMDB API"""
    try:
        encoded_title = quote(movie_title)
        search_url = f"{TMDB_BASE_URL}/search/movie?api_key={TMDB_API_KEY}&query={encoded_title}"
        if year:
            search_url += f"&year={year}"

        response = requests.get(search_url)
        response.raise_for_status()
        data = response.json()

        if data.get("results"):
            for movie in data["results"]:
                if movie.get("poster_path"):
                    return f"{POSTER_BASE_URL}{movie['poster_path']}"
    except Exception as e:
        print(f"Error fetching poster for {movie_title}: {str(e)}")
    return None

def movie_card(movie_title, genres, rating, year=None):
    """Create a styled movie card with poster"""
    poster_url = get_movie_poster(movie_title, year)
    
    card = f"""
    <div style="
        display: flex; 
        background: #2d2d2d;
        border-radius: 10px;
        overflow: hidden;
        margin: 10px 0;
        box-shadow: 0 4px 8px rgba(0,0,0,0.2);
        height: 150px;
        width: 100%;
    ">
    """
    
    if poster_url:
        card += f"""
        <img src="{poster_url}" style="
            height: 150px; 
            width: 100px; 
            object-fit: cover;
        ">
        """
    else:
        card += """
        <div style="
            height: 150px; 
            width: 100px; 
            background: #1f1f1f; 
            display: flex; 
            align-items: center; 
            justify-content: center;
        ">
            <span style="color: #555; font-size: 12px;">No poster</span>
        </div>
        """
    
    card += f"""
        <div style="padding: 15px; flex-grow: 1;">
            <h3 style="margin: 0; color: #fff; font-weight: 600;">{movie_title}</h3>
            <p style="margin: 5px 0; color: #aaa; font-size: 14px;">{genres}</p>
            <div style="display: flex; align-items: center; margin-top: 10px;">
                <span style="color: #ffb700; font-weight: bold;">⭐ {rating:.1f}</span>
    """
    
    if year:
        card += f"""
                <span style="margin-left: 15px; color: #888; font-size: 13px;">{year}</span>
        """
    
    card += """
            </div>
        </div>
    </div>
    """
    
    return card

def create_recommender_interface(loaded_recommender):
    """Create Gradio interface for the NCF recommender"""
    try:
        # Get movie titles from the recommender's data
        movie_options = loaded_recommender.movie_features["movie_title"].dropna().unique().tolist()
        if not movie_options:
            movie_options = ["No movies available"]
    except Exception as e:
        print(f"Error loading movie options: {str(e)}")
        movie_options = ["Error loading movies"]

    with gr.Blocks(
        theme=gr.themes.Default(
            primary_hue="orange",
            secondary_hue="amber",
            neutral_hue="slate",
            font=[gr.themes.GoogleFont("Poppins")]
        ),
        title="Movie Recommender",
        css="""
        .tab-item { padding: 10px 20px !important; }
        .search-box { margin-bottom: 20px !important; }
        .list-view { margin-top: 20px !important; }
        """
    ) as interface:
        
        gr.Markdown("""
        <div style="text-align: center;">
            <h1 style="color: #ffb700;">🎬 Movie Recommender</h1>
            <p style="color: #aaa;">Discover your next favorite movie</p>
        </div>
        """)
        
        # Search bar for user ID
        with gr.Row():
            user_search = gr.Textbox(
                label="Enter User ID",
                placeholder="Search for a user...",
                elem_classes="search-box"
            )
            search_btn = gr.Button("Search", variant="primary")
        
        # Navigation tabs
        with gr.Tabs():
            with gr.TabItem("For You", elem_classes="tab-item"):
                # Personalized recommendations
                with gr.Column(elem_classes="list-view"):
                    gr.Markdown("### Recommended For You")
                    user_recs_output = gr.HTML()
                
                # Popular recommendations
                with gr.Column(elem_classes="list-view"):
                    gr.Markdown("### Popular Movies")
                    popular_recs_output = gr.HTML()
            
            with gr.TabItem("Similar Movies", elem_classes="tab-item"):
                # Similar movies section
                with gr.Row():
                    with gr.Column():
                        movie_dropdown = gr.Dropdown(
                            label="Select Movie",
                            choices=movie_options,
                            value=movie_options[0] if movie_options else None
                        )
                        similar_num = gr.Slider(
                            label="Number of Similar Movies",
                            minimum=1,
                            maximum=20,
                            step=1,
                            value=5
                        )
                        similar_btn = gr.Button("Find Similar", variant="primary")
                    with gr.Column():
                        similar_output = gr.HTML()
        
        # Event handlers
        search_btn.click(
            fn=lambda uid: generate_user_recommendations(loaded_recommender, uid, 5),
            inputs=[user_search],
            outputs=user_recs_output
        )
        
        # Load popular recommendations on startup
        interface.load(
            fn=lambda: get_popular_recommendations(loaded_recommender, 5),
            outputs=popular_recs_output
        )
        
        similar_btn.click(
            fn=lambda movie, num: find_similar_movies(loaded_recommender, movie, num),
            inputs=[movie_dropdown, similar_num],
            outputs=similar_output
        )
    
    return interface

def generate_user_recommendations(loaded_recommender, user_id, num_recs=5):
    """Generate recommendations for a user"""
    try:
        user_id = int(user_id)
        if user_id < 1:
            return "❌ Please enter a valid User ID (positive integer)"
        
        recs = loaded_recommender.recommend_for_user(user_id=user_id, top_n=num_recs)
        
        if recs is None or recs.empty:
            return "❌ No recommendations found for this user"
        
        result = "<div style='display: flex; flex-direction: column; gap: 15px;'>"
        for _, row in recs.iterrows():
            result += movie_card(
                row['title'],
                row.get('genres', ''),
                float(row.get('average_rating', 0)),
                row.get('year', None)
            )
        result += "</div>"
        return result
        
    except ValueError:
        return "❌ Please enter a valid User ID (numbers only)"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def get_popular_recommendations(loaded_recommender, num_recs=5):
    """Get popular movie recommendations"""
    try:
        popular = loaded_recommender.get_popular_recommendations(top_n=num_recs)
        
        if popular is None or popular.empty:
            return "❌ No popular recommendations available"
        
        result = "<div style='display: flex; flex-direction: column; gap: 15px;'>"
        for _, row in popular.iterrows():
            result += movie_card(
                row['title'],
                row.get('genres', ''),
                float(row.get('average_rating', 0)),
                row.get('movie_year', None)
            )
        result += "</div>"
        return result
        
    except Exception as e:
        return f"❌ Error loading popular movies: {str(e)}"

def find_similar_movies(loaded_recommender, movie_title, num_recs):
    """Find similar movies"""
    try:
        movie_data = loaded_recommender.movie_features
        movie_match = movie_data[movie_data["movie_title"].str.strip().str.lower() == movie_title.strip().lower()]
        
        if len(movie_match) == 0:
            return f"❌ Movie '{movie_title}' not found in database"
        
        movie_id = movie_match.index[0]
        similar = loaded_recommender.get_similar_movies(movie_id=movie_id, top_n=num_recs)
        
        if similar is None or similar.empty:
            return f"❌ No similar movies found for {movie_title}"
        
        result = f"<div style='display: flex; flex-direction: column; gap: 15px;'>"
        for _, row in similar.iterrows():
            result += movie_card(
                row['movie_title'],
                row.get('genres', ''),
                float(row.get('movies_avg_rating', 0)),
                row.get('movie_year', None)
            )
        result += "</div>"
        return result
        
    except Exception as e:
        return f"❌ Error finding similar movies: {str(e)}"

In [ ]:
# from pyngrok import ngrok

# if __name__ == "__main__":
#     interface = create_recommender_interface(loaded_recommender)
#     public_url = ngrok.connect(7860)  # Forward port 7860
#     print("Public URL (Share this):", public_url)
#     interface.launch()

In [ ]:
# Usage example
if __name__ == "__main__":
    # Create and launch interface
    interface = create_recommender_interface(loaded_recommender)
    interface.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


In [ ]:
data_processor.df.sample(5)

,Unnamed: 0.1,Unnamed: 0,userId,movieId,user_rating,movie_title,user_rating_count,user_rating_avg,genres,(no genres listed),...,title_90,title_91,title_92,title_93,title_94,title_95,title_96,title_97,title_98,title_99
312020,312020,312020,18162,2907,2,Superstar,32,2.750000,Comedy,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
926609,926611,926611,298041,2676,3,Instinct,33,3.151515,Drama|Thriller,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
627345,627347,627347,148007,2458,2,Armed and Dangerous,35,3.342857,Comedy|Crime,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
649337,649339,649339,33144,99992,2,Shadow Dancer,92,2.619565,Crime|Drama|Thriller,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217283,217283,217283,16022,1727,1,"Horse Whisperer, The",35,3.000000,Drama|Romance,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
